# A09: Bagging

In [ ]:
# Librerías
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from scipy.stats import norm

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from scipy.stats import norm

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split

In [12]:

data = pd.read_csv("Defaultt.csv")

# Preparamos los datos
# Asumimos que tienes 'balance', 'income' y 'student' como predictores
X = data[["balance", "income"]].copy()
X["student"] = data["student"].apply(lambda v: 1 if v == "Yes" else 0)
X = X.values 
y = (data["default"] == "Yes").astype(int).values

# Dividimos Train/Test (Digamos que de tus 10,000 dejas 8,000 para entrenar y 2,000 para validar)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- AQUÍ ESTÁ TU CONFIGURACIÓN EXACTA ---
n_bagging = 500           # 500 Modelos
n_bootstrap_rows = 5000   # 5,000 filas escogidas al azar
n_features_select = 2     # 2 columnas escogidas al azar

n_train_avail = len(y_train)    # Filas disponibles para elegir (ej. 8000)
n_features_total = X.shape[1]   # Columnas disponibles (3)
n_test_samples = len(y_test)

predictions_matrix = np.zeros((n_bagging, n_test_samples))

print(f"Entrenando {n_bagging} árboles.")
print(f"Cada árbol usará: {n_bootstrap_rows} filas y {n_features_select} columnas.")

# 3. El Ciclo
for i in range(n_bagging):
    
    # A. FILAS: Elegir 5000 índices de las filas disponibles CON reemplazo
    # Nota el cambio: size=n_bootstrap_rows (5000)
    row_indices = np.random.choice(range(n_train_avail), size=n_bootstrap_rows, replace=True)
    
    # B. COLUMNAS: Elegir 2 índices de columnas SIN reemplazo
    col_indices = np.random.choice(range(n_features_total), size=n_features_select, replace=False)
    
    # C. Filtrar datos (Filas seleccionadas + Columnas seleccionadas)
    X_sample = X_train[row_indices][:, col_indices]
    y_sample = y_train[row_indices]
    
    # D. Entrenar
    model = DecisionTreeClassifier(random_state=None)
    model.fit(X_sample, y_sample)
    
    # E. Predecir (Usando SOLO las columnas que eligió este árbol específico)
    X_test_subset = X_test[:, col_indices]
    predictions_matrix[i, :] = model.predict(X_test_subset)

# 4. Aggregating
votes_average = np.mean(predictions_matrix, axis=0)
final_predictions = (votes_average >= 0.5).astype(int)

# 5. Resultado
accuracy = np.mean(final_predictions == y_test)
print("------------------------------------------------")
print(f"Exactitud Final: {accuracy:.4f}")

Entrenando 500 árboles.
Cada árbol usará: 5000 filas y 2 columnas.
------------------------------------------------
Exactitud Final: 0.9695


## Regresión logística para comparar

- Accuracy LogisticRegression
- Accuracy Bagging
- Comparación
- Conclusiones

In [ ]:
X_multi = data[["balance", "income"]]
X_multi["student"] = data["student"].apply(lambda v: 1 if v == "Yes" else 0)
y_multi = (data["default"] == "Yes").astype(int)
model_multi = LogisticRegression()
model_multi.fit(X_multi, y_multi)
accuracy_logistic = model_multi.score(X_multi, y_multi)
print("accuracy logistic"+str(accuracy_logistic))

accuracy logistic0.9732


C:\Users\dayan\AppData\Local\Temp\ipykernel_5576\1406709769.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_multi["student"] = data["student"].apply(lambda v: 1 if v == "Yes" else 0)
